Get all comments from TikTok AFD:

In [42]:

import requests
import pandas as pd

 
access_token = get_access_token()


# Specify the fields you want to retrieve
fields = 'text,create_time'

# Specify the video IDs for which you want to retrieve comments
video_ids = ['7355867865917967648',
             '7356904075679550753',
             '7358389091450342689',
             '7356560129510116640',
             '7360616932975938849',
             '7361407002930695457',
             '7363640718352534817',
             '7361739045468327200',
             '7362143776363580704',
             '7349124985140546849',
             '7348761818069781792',
             '7348453458099440929',
             '7347644685818613024',
             '7346599964316880160',
             '7346527520889572641',
             '7346144541419916577',
             '7345786340522331424',
             '7345499661966347552',
             '7345416077377064224',
             '7345103501309005089',
             '7341398475826793761',
             '7340200000674139424',
             '7339869169065692448',
             '7338740558724156704',
             '7338448559542570273',
             '7338371571804867873',
             '7337701981378628897',
             '7337625541001989408',
             '7337269951809522977',
             '7332840390296341792',
             '7332406628865690913',
             '7330978653230189857',
             '7330630868735741216',
             '7330256100383542560',
             '7329876345134042400',
             '7329811216140913953',
             '7328362322886544673',
             '7327639258414501152',
             '7327249454162038049',
             '7326983100666285344',
             '7326902469022289184',
             '7325732224957975841',
             '7325464438104952097',
             '7325010721106971936',
             '7324717706698050848',
             '7324672319329160481',
             '7322065785554357537'] # Add more video IDs as needed

# List to store all comments
all_data = []

# Iterate over each video ID
for video_id in video_ids:
    # Make the request to retrieve comments for the current video ID
    isFetchingComments = True
    cursor = 0
    MAX_COUNT = 100
    while isFetchingComments:
        response = requests.post(
            'https://open.tiktokapis.com/v2/research/video/comment/list/?fields=%s' % fields,
            headers={'authorization': 'bearer ' + access_token},
            data={ 
                'video_id': video_id,
                'max_count': MAX_COUNT,
                'cursor': cursor
            }
        )
        
        # Extract comments from the response
        data = response.json()
        comments = data.get('data', {}).get('comments', [])
        
        # Append comments to the list
        for comment in comments:
            comment['video_id'] = video_id
            all_data.append(comment)
        
        if len(comments) == 0:
            isFetchingComments = False
            
        else:
            cursor += MAX_COUNT

# Create DataFrame
df = pd.DataFrame(all_data)

print(df)



       create_time                                               text  \
0       1712671729  Bin ich froh, daß ihr das wenigstens noch @aus...   
1       1712671788  alle die nicht arbeiten/die die arbeiten und v...   
2       1712673774                                              💙💙💯👍💯   
3       1712671590                           Aber zügig weg mit denen   
4       1712671658                                              💙💙💙💙💙   
...            ...                                                ...   
21949   1707141992                                                      
21950   1707690107                                          caouillo😭   
21951   1707207523  Echt, wir brauchen Landwirtschaft ? Welch neue...   
21952   1707665939                                                      
21953   1708792875                                                      

                  video_id  
0      7355867865917967648  
1      7355867865917967648  
2      7355867865917967648  
3      

In [43]:
#### CONVERT DATE SO IT IS LATER READABLE IN R:

import pandas as pd

# Step 1: Create a sample DataFrame

df = pd.DataFrame(all_data)

# Step 2: Convert the Unix timestamps to human-readable date and time
df['create_time_readable'] = pd.to_datetime(df['create_time'], unit='s')

# Display the DataFrame
print(df)


       create_time                                               text  \
0       1712671729  Bin ich froh, daß ihr das wenigstens noch @aus...   
1       1712671788  alle die nicht arbeiten/die die arbeiten und v...   
2       1712673774                                              💙💙💯👍💯   
3       1712671590                           Aber zügig weg mit denen   
4       1712671658                                              💙💙💙💙💙   
...            ...                                                ...   
21949   1707141992                                                      
21950   1707690107                                          caouillo😭   
21951   1707207523  Echt, wir brauchen Landwirtschaft ? Welch neue...   
21952   1707665939                                                      
21953   1708792875                                                      

                  video_id create_time_readable  
0      7355867865917967648  2024-04-09 14:08:49  
1      7355867865917967

In [44]:
#Check working directory
import os

# Check current working directory
print(os.getcwd())

# Save to the current working directory
df.to_csv('AFD_comments.csv', index=False)


/Users/emma/Desktop/FINAL DATA/DATA_retrieval


CONDUCT SENTIMENT ANALYSIS ON ALL COMMENTS OF AFD:

In [1]:
# This is our classification prompt
PROMPT =  "Conduct a sentiment analysis of the comments on TikTok Vidoes by the official social media profiles of German parliamentarian parties on TikTok. The comments are generated in German. Consider the tone, context, and language and emojis used in each comment to determine its sentiment.  Respond with '0' if the comment contains neutral sentiment, respond with '1' if the content contains positive sentiment, and '2' if the comment contains negative sentiments. No explanation is needed. Here are some examples to guide your analysis. The comment 'Das nenn ich mal eine gute Partei',  the sentiment should be categorized as positive, so 1. If the comment is 'Ich habe keine starken Gefühle zu diesem Thema', the sentiment should be catgeorized as neutral, so 0. An example for a negative sentiment comment is 'Wenn das böse ein Gesicht hat', so it should be categorizes as 2." 

In [2]:
import openai
from openai import OpenAI

In [3]:
#We define which model to use throughout
MODEL = 'gpt-3.5-turbo-0125' # We use the cheap model 
WAIT_TIME = 0.01 # Wait time between each request. This depends on the rate limit of the model used

client = OpenAI(
    api_key= "" #Set the API key. See the how-to guide for further instructions
)

In [4]:
# Simple function for analyzing a message with OpenAI
def analyze_message(text, temperature=0.2):
    try:
        response = client.chat.completions.create(
            model = MODEL, 
            temperature=temperature,
            messages=[
                    {"role": "system", "content": f"'{PROMPT}'"}, #The system instruction tells the bot how it is supposed to behave
                    {"role": "user", "content": f"'{text}'"} #This provides the text to be analyzed.
                ]
        )
        result = ''
        for choice in response.choices:
            result += choice.message.content
        print(f"{text}: {result}")
        return result 
    except Exception as e:
        # Not the prettiest solution; but easy
        print(f'Exception happened: {e}')
        return 'error'

In [5]:
analyze_message("'Euch würde ich nicht wählen!'")

'Euch würde ich nicht wählen!': 2


'2'

In [6]:
import pandas as pd

# Specify the path to the CSV file
file_path = '/Users/emma/Desktop/cleaned_data.csv'

# Load the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Display the first few rows of the DataFrame
print(df)

       create_time                                               text  \
0       1712671729  Bin ich froh, daß ihr das wenigstens noch @aus...   
1       1712671788  alle die nicht arbeiten/die die arbeiten und v...   
2       1712673774                                              💙💙💯👍💯   
3       1712671590                           Aber zügig weg mit denen   
4       1712671658                                              💙💙💙💙💙   
...            ...                                                ...   
11912   1706292746                       Toll nicht schwer zu kriegen   
11913   1706518310                                                💙💙💙   
11914   1706523057                              Ich wähle die AfD 💙💙💙   
11915   1707690107                                          caouillo😭   
11916   1707207523  Echt, wir brauchen Landwirtschaft ? Welch neue...   

           video_id create_time_readable  
0      7.355868e+18  2024-04-09 14:08:49  
1      7.355868e+18  2024-04-09 14:09

In [8]:
# We start by defining a new column with the result
df['sentiment'] = None

# Save the DataFrame to a CSV file
df.to_csv('/Users/emma/SEN_AFD.csv', compression='gzip', index=False)

In [9]:
import time 
i = 0
while(True):

    #Find all unprocessed lines; where 
    left = df.loc[df['sentiment'].isna()]
    
    #No lines left? Then we're done
    if len(left)==0:
        print("All done!")
        break
        
    #Take a random line
    line = left.sample()
    index = line.index.values[0]
    text = line['text'].values[0]
    
    print(f"There are {len(left)} left to process. Processing: {index}")
    
    #Wait for a bit, to not overload the API
    time.sleep(WAIT_TIME)
    
    #Analyze the specific line, chunk by chunk    
    result = analyze_message(text)
    df.loc[index,'sentiment'] = result 

    i+=1    
    #Save the result to persistent file
    df.to_csv('/Users/emma/Sentiment_AFD.csv.gz',compression='gzip',index=False)

There are 11917 left to process. Processing: 5749
💙💙💙: 1
There are 11916 left to process. Processing: 9162
💙💙🥰: 1
There are 11915 left to process. Processing: 2990
versteh deinen Post nicht,sie haben auch Recht behalten: 2
There are 11914 left to process. Processing: 11043
daran ist falsch dass viele davon hier aufgewaschen sind & das unser zuhause geworden ist, welches wir jetzt verlassen müssen: 2
There are 11913 left to process. Processing: 1714
Kein Krieg mehr.: 1
There are 11912 left to process. Processing: 9444
Außerdem vergisst er zu sagen, dass mit Remigration die gewalttätige Abschiebung von Deutschen gemeint ist!: 2
There are 11911 left to process. Processing: 1894
☝️🇩🇪💙💙💙: 1
There are 11910 left to process. Processing: 4404
Ja genau die haben Realitätssinn: 1
There are 11909 left to process. Processing: 2420
Was regen sich jetzt alle so auf ? Der Großteil der im Minutentakt verblödenden auf Linie gebrachten deutschen Bevölkerung hat bewusst das so gewählt: 2
There are 11908 

In [10]:
#Check if data is the way we want
print(df.head(10))

   create_time                                               text  \
0   1712671729  Bin ich froh, daß ihr das wenigstens noch @aus...   
1   1712671788  alle die nicht arbeiten/die die arbeiten und v...   
2   1712673774                                              💙💙💯👍💯   
3   1712671590                           Aber zügig weg mit denen   
4   1712671658                                              💙💙💙💙💙   
5   1712671708                                           💙💙💙💙💙💙💙💙   
6   1712673543                                Absolut korrekt 💙💙💙   
7   1712671754                                             💙💙💙💙💙💙   
8   1712677943                                       Liebe euch 🫶   
9   1712677358  Alleine als Mann gehe ich abends auch nicht ra...   

       video_id create_time_readable sentiment  
0  7.355868e+18  2024-04-09 14:08:49         1  
1  7.355868e+18  2024-04-09 14:09:48         2  
2  7.355868e+18  2024-04-09 14:42:54         1  
3  7.355868e+18  2024-04-09 14:06:30        

In [11]:
# Export to CSV
df.to_csv('/Users/emma/AFD_done.csv',index=False)